In [7]:
data = spark.read.format("csv").option("header",True).option("inferschema",True).load("file:/home/hduser/insurance.csv")
data.show()

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
|abc|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|
| 37|  male| 29.83|       2|    no|northeast|  6406.4107|
| 60|female| 25.84|       0|    no|northwest|28923.13692|
| 25|  male| 26.22|       0|    no|northeast|  2721.3208|
| 62|female| 26.29|       0|   yes|southeast| 27808.7251|
| 23|  male|  34.4|       0|    no|southwest|   1826.843|
| 56|female| 39.82|       0|    no|southeast| 11090.7178|
| 27|  male| 4

In [9]:
data.printSchema()

root
 |-- age: string (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)



In [12]:
data.describe().show()

+-------+-----------------+------+------------------+-----------------+------+---------+------------------+
|summary|              age|   sex|               bmi|         children|smoker|   region|           charges|
+-------+-----------------+------+------------------+-----------------+------+---------+------------------+
|  count|             1338|  1338|              1338|             1338|  1338|     1338|              1338|
|   mean|39.21540762902019|  null|30.663396860986538|  1.0949177877429|  null|     null|13270.422265141257|
| stddev|14.05187040320521|  null| 6.098186911679012|1.205492739781914|  null|     null|12110.011236693992|
|    min|               18|female|             15.96|                0|    no|northeast|         1121.8739|
|    max|              abc|  male|             53.13|                5|   yes|southwest|       63770.42801|
+-------+-----------------+------+------------------+-----------------+------+---------+------------------+



In [25]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
data1 = data.withColumn("age_c", when(data["age"] == "abc","20").otherwise(data["age"]).cast(IntegerType()))

data2 = data1.drop("age")

data3 = data2.withColumnRenamed("age_c","age")

data3.printSchema()


root
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)
 |-- age: integer (nullable = true)



In [36]:
from pyspark.ml.feature import StringIndexer
category = StringIndexer(inputCol = "sex", outputCol = "gender_categorical")
categorised = category.fit(data3).transform(data3)
categorised.show()


+------+------+--------+------+---------+-----------+---+------------------+
|   sex|   bmi|children|smoker|   region|    charges|age|gender_categorical|
+------+------+--------+------+---------+-----------+---+------------------+
|female|  27.9|       0|   yes|southwest|  16884.924| 19|               1.0|
|  male| 33.77|       1|    no|southeast|  1725.5523| 18|               0.0|
|  male|  33.0|       3|    no|southeast|   4449.462| 20|               0.0|
|  male|22.705|       0|    no|northwest|21984.47061| 33|               0.0|
|  male| 28.88|       0|    no|northwest|  3866.8552| 32|               0.0|
|female| 25.74|       0|    no|southeast|  3756.6216| 31|               1.0|
|female| 33.44|       1|    no|southeast|  8240.5896| 46|               1.0|
|female| 27.74|       3|    no|northwest|  7281.5056| 37|               1.0|
|  male| 29.83|       2|    no|northeast|  6406.4107| 37|               0.0|
|female| 25.84|       0|    no|northwest|28923.13692| 60|               1.0|

In [38]:
category = StringIndexer(inputCol = "smoker", outputCol = "smoker_categorical")
categorised = category.fit(categorised).transform(categorised)
categorised.show()

+------+------+--------+------+---------+-----------+---+------------------+------------------+------------------+
|   sex|   bmi|children|smoker|   region|    charges|age|gender_categorical|region_categorical|smoker_categorical|
+------+------+--------+------+---------+-----------+---+------------------+------------------+------------------+
|female|  27.9|       0|   yes|southwest|  16884.924| 19|               1.0|               1.0|               1.0|
|  male| 33.77|       1|    no|southeast|  1725.5523| 18|               0.0|               0.0|               0.0|
|  male|  33.0|       3|    no|southeast|   4449.462| 20|               0.0|               0.0|               0.0|
|  male|22.705|       0|    no|northwest|21984.47061| 33|               0.0|               2.0|               0.0|
|  male| 28.88|       0|    no|northwest|  3866.8552| 32|               0.0|               2.0|               0.0|
|female| 25.74|       0|    no|southeast|  3756.6216| 31|               1.0|    

In [63]:
category = StringIndexer(inputCol = "region", outputCol = "region_categorical")
categorised = category.fit(categorised).transform(categorised)


AttributeError: 'StringIndexer' object has no attribute 'transform'

In [44]:
cleands = categorised.drop("sex","smoker","region")
cleands.show()
cleands.printSchema()

+------+--------+-----------+---+------------------+------------------+------------------+
|   bmi|children|    charges|age|gender_categorical|region_categorical|smoker_categorical|
+------+--------+-----------+---+------------------+------------------+------------------+
|  27.9|       0|  16884.924| 19|               1.0|               1.0|               1.0|
| 33.77|       1|  1725.5523| 18|               0.0|               0.0|               0.0|
|  33.0|       3|   4449.462| 20|               0.0|               0.0|               0.0|
|22.705|       0|21984.47061| 33|               0.0|               2.0|               0.0|
| 28.88|       0|  3866.8552| 32|               0.0|               2.0|               0.0|
| 25.74|       0|  3756.6216| 31|               1.0|               0.0|               0.0|
| 33.44|       1|  8240.5896| 46|               1.0|               0.0|               0.0|
| 27.74|       3|  7281.5056| 37|               1.0|               2.0|               0.0|

In [51]:
from pyspark.ml.feature import VectorAssembler

featuredata  = VectorAssembler(inputCols=["age","bmi", "children", "gender_categorical", "smoker_categorical", "region_categorical"],outputCol="features")
results = featuredata.transform(cleands)
results.select("features").show(truncate=False)
results.printSchema()


+-----------------------------+
|features                     |
+-----------------------------+
|[19.0,27.9,0.0,1.0,1.0,1.0]  |
|[18.0,33.77,1.0,0.0,0.0,0.0] |
|[20.0,33.0,3.0,0.0,0.0,0.0]  |
|[33.0,22.705,0.0,0.0,0.0,2.0]|
|[32.0,28.88,0.0,0.0,0.0,2.0] |
|[31.0,25.74,0.0,1.0,0.0,0.0] |
|[46.0,33.44,1.0,1.0,0.0,0.0] |
|[37.0,27.74,3.0,1.0,0.0,2.0] |
|[37.0,29.83,2.0,0.0,0.0,3.0] |
|[60.0,25.84,0.0,1.0,0.0,2.0] |
|[25.0,26.22,0.0,0.0,0.0,3.0] |
|[62.0,26.29,0.0,1.0,1.0,0.0] |
|[23.0,34.4,0.0,0.0,0.0,1.0]  |
|[56.0,39.82,0.0,1.0,0.0,0.0] |
|[27.0,42.13,0.0,0.0,1.0,0.0] |
|[19.0,24.6,1.0,0.0,0.0,1.0]  |
|[52.0,30.78,1.0,1.0,0.0,3.0] |
|[23.0,23.845,0.0,0.0,0.0,3.0]|
|[56.0,40.3,0.0,0.0,0.0,1.0]  |
|[30.0,35.3,0.0,0.0,1.0,1.0]  |
+-----------------------------+
only showing top 20 rows

root
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- charges: double (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender_categorical: double (nullable = false)

In [53]:
for_model = results.select("features", "charges")
for_model.show(truncate=False)

+-----------------------------+-----------+
|features                     |charges    |
+-----------------------------+-----------+
|[19.0,27.9,0.0,1.0,1.0,1.0]  |16884.924  |
|[18.0,33.77,1.0,0.0,0.0,0.0] |1725.5523  |
|[20.0,33.0,3.0,0.0,0.0,0.0]  |4449.462   |
|[33.0,22.705,0.0,0.0,0.0,2.0]|21984.47061|
|[32.0,28.88,0.0,0.0,0.0,2.0] |3866.8552  |
|[31.0,25.74,0.0,1.0,0.0,0.0] |3756.6216  |
|[46.0,33.44,1.0,1.0,0.0,0.0] |8240.5896  |
|[37.0,27.74,3.0,1.0,0.0,2.0] |7281.5056  |
|[37.0,29.83,2.0,0.0,0.0,3.0] |6406.4107  |
|[60.0,25.84,0.0,1.0,0.0,2.0] |28923.13692|
|[25.0,26.22,0.0,0.0,0.0,3.0] |2721.3208  |
|[62.0,26.29,0.0,1.0,1.0,0.0] |27808.7251 |
|[23.0,34.4,0.0,0.0,0.0,1.0]  |1826.843   |
|[56.0,39.82,0.0,1.0,0.0,0.0] |11090.7178 |
|[27.0,42.13,0.0,0.0,1.0,0.0] |39611.7577 |
|[19.0,24.6,1.0,0.0,0.0,1.0]  |1837.237   |
|[52.0,30.78,1.0,1.0,0.0,3.0] |10797.3362 |
|[23.0,23.845,0.0,0.0,0.0,3.0]|2395.17155 |
|[56.0,40.3,0.0,0.0,0.0,1.0]  |10602.385  |
|[30.0,35.3,0.0,0.0,1.0,1.0]  |3

In [57]:
train_data, test_data = for_model.randomSplit([0.7,0.3])
train_data.describe().show()

+-------+------------------+
|summary|           charges|
+-------+------------------+
|  count|               939|
|   mean|13295.366703598504|
| stddev|11904.182451723105|
|    min|         1121.8739|
|    max|       63770.42801|
+-------+------------------+



In [59]:
test_data.describe().show()

+-------+------------------+
|summary|           charges|
+-------+------------------+
|  count|               399|
|   mean|13211.718436290725|
| stddev|12596.232103513656|
|    min|         1131.5066|
|    max|       62592.87309|
+-------+------------------+



In [64]:
from pyspark.ml.regression import LinearRegression

lr_model = LinearRegression(featuresCol= "features",labelCol="charges")
training_model = lr_model.fit(train_data)
result = training_model.transform(test_data)
result.show()

+--------------------+-----------+------------------+
|            features|    charges|        prediction|
+--------------------+-----------+------------------+
|(6,[0,1],[18.0,30...|  1131.5066|1966.6228005545763|
|(6,[0,1],[18.0,34...|   1137.011| 3332.284045139075|
|(6,[0,1],[18.0,34...|  1137.4697|3446.0891488544494|
|(6,[0,1],[18.0,41...|  1146.7966| 5760.126257733738|
|(6,[0,1],[18.0,43...|  1149.3959|6405.0218454541955|
|(6,[0,1],[18.0,53...|  1163.4627| 9895.045026059028|
|(6,[0,1],[21.0,23...|  1515.3449| 292.0518631349787|
|(6,[0,1],[23.0,41...|  1837.2819|7217.8985674084015|
|(6,[0,1],[29.0,27...|  2867.1196| 3830.821657997416|
|(6,[0,1],[41.0,40...|  5709.1644|10940.890492451172|
|(6,[0,1],[50.0,25...|   8442.667| 7927.734512497334|
|(6,[0,1],[53.0,29...|  9487.6442| 10084.60206682869|
|(6,[0,1],[53.0,31...|27346.04207| 10729.49765454915|
|(6,[0,1],[53.0,41...|  9504.3103|14219.520835153975|
|(6,[0,1],[55.0,38...| 10226.2842|13596.295659640868|
|(6,[0,1],[57.0,40...| 10982

In [66]:
output = training_model.evaluate(train_data)
print(output.r2)
print(output.meanSquaredError)

0.7426737892155811
36426749.57873799


In [68]:
output = training_model.evaluate(test_data)
print(output.r2)
print(output.meanSquaredError)

0.7656085289946948
37096530.199596964


In [70]:
training_model.write().overwrite().save('file:/home/hduser/insurancepredictor')
print("Model Saved")

Model Saved


In [74]:
from pyspark.ml.regression import LinearRegressionModel
insurnace_model = LinearRegressionModel.load('file:/home/hduser/insurancepredictor')
res = insurnace_model.transform(test_data)
res.show()

+--------------------+-----------+------------------+
|            features|    charges|        prediction|
+--------------------+-----------+------------------+
|(6,[0,1],[18.0,30...|  1131.5066|1966.6228005545763|
|(6,[0,1],[18.0,34...|   1137.011| 3332.284045139075|
|(6,[0,1],[18.0,34...|  1137.4697|3446.0891488544494|
|(6,[0,1],[18.0,41...|  1146.7966| 5760.126257733738|
|(6,[0,1],[18.0,43...|  1149.3959|6405.0218454541955|
|(6,[0,1],[18.0,53...|  1163.4627| 9895.045026059028|
|(6,[0,1],[21.0,23...|  1515.3449| 292.0518631349787|
|(6,[0,1],[23.0,41...|  1837.2819|7217.8985674084015|
|(6,[0,1],[29.0,27...|  2867.1196| 3830.821657997416|
|(6,[0,1],[41.0,40...|  5709.1644|10940.890492451172|
|(6,[0,1],[50.0,25...|   8442.667| 7927.734512497334|
|(6,[0,1],[53.0,29...|  9487.6442| 10084.60206682869|
|(6,[0,1],[53.0,31...|27346.04207| 10729.49765454915|
|(6,[0,1],[53.0,41...|  9504.3103|14219.520835153975|
|(6,[0,1],[55.0,38...| 10226.2842|13596.295659640868|
|(6,[0,1],[57.0,40...| 10982